In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import re
from datetime import datetime
import base64
import hashlib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Caminho para o arquivo xlsx local
file_path = './banco_dados.xlsx'

In [ ]:
# Leitura do arquivo xlsx usando pandas
df = pd.read_excel(file_path, sheet_name='BDados')

# Inicializa variáveis necessárias
dados = {}
count = 0
timeAnterior = []  # Inicializa timeAnterior como uma lista vazia

# Loop para processar colunas que começam com "Tela"
for indexCol, colStr in enumerate(df.columns):
    if colStr.startswith("Tela") and df.iloc[0, indexCol] != "" and pd.notna(df.iloc[0, indexCol]):
        timeAtual = []
        timeElipsed = []
        value = []
        
        colNum = int(re.search(r"Tela (\d+)", colStr).group(1))
        
        for indexRow in range(df.shape[0]):
            cell_value = str(df.iloc[indexRow, indexCol])
            match = re.match(r"(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d{3}) - (.+)", cell_value)
            
            if match:
                timeAtual.append(pd.to_datetime(match.group(1)))
                
                if colNum != 1:
                    valores = match.group(2)
                    valores = re.sub(r'<optimized out>#\w+\((Sim|Não)\)', r'\1', valores)
                    valores = valores.replace("other;", "").replace("Sucess;", "").replace("Sucess", "")
                    valueAux = valores.split(";") 
                    timeElipsedAux = [(timeAtual[-1] - timeAnterior[indexRow]).total_seconds()]
                    if len(valueAux) == 1:
                        timeElipsed += timeElipsedAux
                        value += valueAux
                    else:
                        timeElipsed += [timeElipsedAux * len(valueAux)]                        
                        value += [valueAux]
            else:
                print("Ocorreu um erro ao processar a célula:", cell_value)
        
        timeAnterior = timeAtual
        
        if colNum != 1:
            if isinstance(value[0], list):
                for index in range(len(value[0])):
                    dados[f"Resp_{count}"] = [sublista[index] for sublista in value if len(sublista) > index]
                    try:
                        dados[f"Elipsed_{count}"] = [sublista[index] for sublista in timeElipsed if len(sublista) > index]
                    except Exception as e:
                        print(f"Ocorreu um erro: {e}")
                    count += 1
            else: 
                dados[f"Resp_{count}"] = value
                dados[f"Elipsed_{count}"] = timeElipsed
                count += 1  

# Cria o novo DataFrame com os dados processados
df_novo = pd.DataFrame(dados)

# Usando ExcelWriter para adicionar a nova aba ao arquivo Excel existente com if_sheet_exists='replace'
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_novo.to_excel(writer, sheet_name='TDados', index=False)